## This notebook is meant to perform contrastive learning between fMRI data and CLIP features on BOLD5000 dataset

In [1]:
import numpy as np
import nibabel as nib
import nilearn 
import matplotlib.pyplot as plt
import os
from os.path import join as opj
import pandas as pd
import seaborn as sns
import glob
from nilearn import plotting
from nilearn.image import *
import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from nilearn.plotting import plot_stat_map
from nilearn.image import mean_img
from nilearn.plotting import plot_img, plot_epi
from nilearn.maskers import NiftiMasker
from sklearn.preprocessing import StandardScaler
import wandb
import pickle
from torch.utils.data import Dataset, DataLoader
from dataset import fMRI_Dataset
import torch
from torch import nn
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint

from pytorch_lightning.loggers import WandbLogger


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


## Load and concatenate dataset for all subjects

In [2]:
## load the data

train_datasets=[]
val_datasets=[]
test_datasets=[]

for subj in tqdm.tqdm(["CSI1","CSI2","CSI3","CSI4"]):

    subj_id = int(subj.split("CSI")[1])

    data_path =  f"/home/matteo/storage/brain_tuning/{subj}"

    train_fmri = np.load(opj(data_path, "train_fmri_top.npy"))
    val_fmri = np.load(opj(data_path, "val_fmri_top.npy"))
    test_fmri = np.load(opj(data_path, "test_fmri_top.npy"))

    ##load the images
    img_train = np.load(opj(data_path, "img_train.npy"),allow_pickle=True)
    img_val = np.load(opj(data_path, "img_val.npy"),allow_pickle=True)
    img_test = np.load(opj(data_path, "img_test.npy"),allow_pickle=True)

    ## load the features
    train_features = np.load(opj(data_path, "train_image_features.npy"))
    val_features = np.load(opj(data_path, "val_image_features.npy"))
    test_features = np.load(opj(data_path, "test_image_features.npy"))

    ## create the dataset
    train_dataset = fMRI_Dataset(train_fmri,img_train,train_features,subj_id)
    val_dataset = fMRI_Dataset(val_fmri,img_val,val_features,subj_id)
    test_dataset = fMRI_Dataset(test_fmri,img_test,test_features,subj_id)

    ## append the datasets
    train_datasets.append(train_dataset)
    val_datasets.append(val_dataset)
    test_datasets.append(test_dataset)





100%|██████████| 4/4 [00:07<00:00,  1.93s/it]


In [3]:
## Concatenate the datasets
train_dataset = torch.utils.data.ConcatDataset(train_datasets)
val_dataset = torch.utils.data.ConcatDataset(val_datasets)
test_dataset = torch.utils.data.ConcatDataset(test_datasets)


In [4]:
BATCH_SIZE = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
## check the shape of the data
output = next(iter(train_loader))
fmri = output["data"]
img = output["images"]
features = output["features"]
subj_id = output["subject_id"]

print(fmri.shape)
print(img.shape)
print(features.shape)
print(subj_id.shape)


torch.Size([256, 10000])
torch.Size([256, 3, 375, 375])
torch.Size([256, 768])
torch.Size([256])


## Load the architecture 

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim,act_fn=nn.ReLU,alignment_layers_keys=[1,2,3,4],common_dim=4096):
        super(Encoder, self).__init__()
        self.common_dim=common_dim
        self.alignment_layers={}
        for k in alignment_layers_keys:
            self.alignment_layers[k]=nn.LazyLinear(common_dim)
        
        
        layers = []
        prev_dim = input_dim
        if len(hidden_dims):
            
            layers.append(nn.LayerNorm(common_dim))
            for hidden_dim in hidden_dims:
                layers.append(nn.LazyLinear(hidden_dim))
                layers.append(act_fn())
                prev_dim = hidden_dim
            
            #another layer norm
            layers.append(nn.LazyLinear(output_dim))
            layers.append(nn.LayerNorm(output_dim))
            
            self.net = nn.Sequential(*layers)     
        else:
            self.net = nn.Linear(input_dim, output_dim)

        
    def _apply(self, fn):
        super(Encoder, self)._apply(fn)        
        for k,v in self.alignment_layers.items():
            self.alignment_layers[k]._apply(fn)
            
    
    def forward(self, x, k=None):
        
        def apply_alignment_layers(x, k, alignment_layers):
            # Create an empty tensor to store the results
            result = torch.empty_like(x)
            result = result[:,:self.common_dim]
            
            # Iterate through each unique key in k
            for key in k.unique():
                # Create a mask for all elements that match the current key
                mask = (k == key.item())
                
                # print(x.shape, result.shape, mask)
                
                # Apply the corresponding alignment layer to the masked elements
                result[mask] = alignment_layers[key.item()](x[mask])
            
            return result
        
        if k is None:
            k=torch.ones(len(x))
        # Apply alignment layers to x using the custom function
        x = apply_alignment_layers(x, k, self.alignment_layers)
        
        # x = self.alignment_layers[k](x)
        return self.net(x)




In [7]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class ContrastiveModel(pl.LightningModule):

    def contrastive_loss(self, z_i, z_j, normalize =True):
        if normalize:
            z_i = nn.functional.normalize(z_i, dim=1)
            z_j = nn.functional.normalize(z_j, dim=1)
        
        logits = (z_i @ z_j.T) / self.temperature
        similarities = z_j @ z_j.T
        # targets = torch.nn.functional.softmax(similarities * self.temperature, dim=-1)

        targets = torch.arange(logits.shape[0]).long().to(logits.device)
        
        loss = 0.5*torch.nn.functional.cross_entropy(logits, targets) + 0.5*torch.nn.functional.cross_entropy(logits.T, targets)
        return loss
    
    def mean_contrastive(self, z_i, z_j, temperature=1.0):
        return (1-self.alpha)*nn.functional.mse_loss(z_i, z_j)+(self.alpha)*self.contrastive_loss(z_i, z_j)
    
    def cosine_loss(self, z_i, z_j, temperature=1.0):
        cosine_similarity = torch.nn.functional.cosine_similarity(z_i, z_j).mean()
        return 1- cosine_similarity

    def __init__(self,  num_input_channels: int, hidden_dims, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU, temperature=.1,loss_type="contrastive",alpha=0.8, lr=1e-4,wd = 1e-4):
        super().__init__()
        
        self.temperature=temperature
        
        self.model = Encoder(num_input_channels, base_channel_size, latent_dim, act_fn)
        self.alpha = alpha
        self.lr = lr
        self.wd = wd

        self.loss_type=loss_type
        if loss_type=="contrastive":
            self.loss_fn=self.contrastive_loss
        elif loss_type=="mean_contrastive":
            self.loss_fn=self.mean_contrastive
        
        elif loss_type=="mse":
            self.loss_fn=torch.nn.functional.mse_loss
        elif loss_type=="cosine":
            self.loss_fn=self.cosine_loss
        
        self.train_losses = []
        self.train_mse=[]
        self.train_cosine=[]
        self.val_losses = []
        self.val_mse=[]
        self.val_cosine=[]
    
        self.train_history={}
        self.val_history={}

        #init history
        self.train_history["train_loss"]=[]
        self.train_history["train_mse"]=[]
        self.train_history["train_cosine"]=[]
        self.val_history["val_loss"]=[]
        self.val_history["val_mse"]=[]
        self.val_history["val_cosine"]=[]


    def forward(self, x, embeddings = None,**kwargs):
        fmri_proj = self.model(x, **kwargs)
        
        if embeddings is not None:
            embeddings_proj = embeddings   ##Possibly add a layer to adapt this?
            return fmri_proj, embeddings_proj
        
        return fmri_proj


        # return self.model(x,**kwargs)
    
    def training_step(self, batch, batch_idx):
        x = batch["data"]
        embeddings = batch["features"]
        idx = batch["subject_id"]

        # x = x.float()
        y_hat_fmri, y_hat_embeddings = self(x, embeddings, k=idx)
        loss = self.loss_fn(y_hat_fmri, y_hat_embeddings)


        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.train_losses.append(loss.item())

        mse_loss = torch.nn.functional.mse_loss(y_hat_fmri, y_hat_embeddings)
        cosine_similarity = torch.nn.functional.cosine_similarity(y_hat_fmri, y_hat_embeddings).mean()
        self.train_mse.append(mse_loss.item())
        self.train_cosine.append(cosine_similarity.item())

        return loss
    
    def validation_step(self, batch, batch_idx):
        
        
        x = batch["data"]
        embeddings = batch["features"]
        idx = batch["subject_id"]

        y_hat_fmri, y_hat_embeddings = self(x, embeddings, k=idx)

        loss = self.loss_fn(y_hat_fmri, y_hat_embeddings)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        
        mse_loss = torch.nn.functional.mse_loss(y_hat_fmri, y_hat_embeddings)
        self.log('val_mse_loss', mse_loss, on_epoch=True, prog_bar=True)
        
        cosine_similarity = torch.nn.functional.cosine_similarity(y_hat_fmri, y_hat_embeddings).mean()
        self.log('val_cosine_similarity', cosine_similarity, on_epoch=True, prog_bar=True)
        
        self.val_losses.append(loss.item())
        self.val_mse.append(mse_loss.item())
        self.val_cosine.append(cosine_similarity.item())
        return mse_loss
        
    
    def on_train_epoch_end(self) -> None:
        self.train_history["train_loss"].append(np.mean(self.train_losses))
        self.train_history["train_mse"].append(np.mean(self.train_mse))
        self.train_history["train_cosine"].append(np.mean(self.train_cosine))
        self.train_losses = []
        self.train_mse=[]
        self.train_cosine=[]
        return super().on_train_epoch_end()
    
    def on_validation_epoch_end(self) -> None:
        self.val_history["val_loss"].append(np.mean(self.val_losses))
        self.val_history["val_mse"].append(np.mean(self.val_mse))
        self.val_history["val_cosine"].append(np.mean(self.val_cosine))
        self.val_losses = []

        self.val_mse=[]
        self.val_cosine=[]
        return super().on_validation_epoch_end()

        
    def configure_optimizers(self):
        # return torch.optim.AdamW(self.parameters(), lr=3e-4, weight_decay=0)
        # add a scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.wd)
        # use a scheduler that every 100 steps, it will reduce the learning rate by 0.1
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=50, verbose=True)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

In [8]:
# Define the sweep configuration
sweep_config = {
    'method': 'random',  # can also be 'random' or 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'BS' : {'values': [512,1024,2048]
               },
        'lr': {
            'values': [1e-4, 1e-5]
        },
        'alpha': {
            'values': [0.5, 0.8]
        },
        'temperature': {
            'values': [0.1]
        },
        'loss_type': {
            'values': ['contrastive', 'mse', 'mean_contrastive']
        },
        'wd': {
            'values': [1e-4, 1e-5,1e-2,0]
        },
        'latent_dim':
            {'values': [768]
            },
        
        'act_fn':
         {'values': ['ReLU', 'Identity']
         },
        
        'base_channel_size': {
            'values': [[4096,2048,1024], [2048,1024], [2048]]
            },
        
        'hidden_dims':
            {'values': [[2048,1024,512], [1024,512], [1024]]
            },
   }
}

# Initialize the wandb sweep
sweep_id = wandb.sweep(sweep_config, project='SweepBrainTuning')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: si2zjyre
Sweep URL: https://wandb.ai/matteoferrante/SweepBrainTuning/sweeps/si2zjyre


## Optimization Loop

In [9]:
## obtain the text and fmri embeddings iterating over the training and test loaders
def get_embeddings(model, loader):
    fmri_embeddings = []
    img_embeddings = []
    for batch in tqdm.tqdm(loader):
        x = batch["data"]
        embeddings = batch["features"]
        idx = batch["subject_id"]
        with torch.no_grad():
            y_hat_fmri, y_hat_embeddings = model(x, embeddings, k=idx)
            fmri_embeddings.append(y_hat_fmri)
            img_embeddings.append(y_hat_embeddings)
    return torch.cat(fmri_embeddings), torch.cat(img_embeddings)


In [10]:
import torch
import wandb
import numpy as np
import tqdm
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

# Assuming the brain_model and other required variables are defined

## EVALUATION FUNCTIONS

def organize_results(loader,brain_model):
    # Initialize the dictionary to store results for each subject
    results = defaultdict(lambda: {"y_pred": [], "y_gt": [], "gt_images": []})

    # Run inference and store predictions and ground truth
    with torch.no_grad():
        for batch in tqdm.tqdm(loader):
            x = batch["data"]
            embeddings = batch["features"]
            k = batch["subject_id"]
            img = batch["images"]

            y_hat_fmri, y_hat_embeddings = brain_model(x, embeddings, k=k)
            for subject_index in torch.unique(k):
                mask = k == subject_index
                results[subject_index.item()]["y_pred"].append(y_hat_fmri[mask])
                results[subject_index.item()]["y_gt"].append(y_hat_embeddings[mask])
                results[subject_index.item()]["gt_images"].extend(img[mask])

    # Concatenate the results for each subject
    for subject_index in results:
        results[subject_index]["y_pred"] = torch.cat(results[subject_index]["y_pred"], dim=0)
        results[subject_index]["y_gt"] = torch.cat(results[subject_index]["y_gt"], dim=0)
    
    return results

# Identification accuracy function
def identification_accuracy_fast(P, T):
    n = P.shape[0]
    if n == 0:
        return np.nan

    P_mean = np.mean(P, axis=1, keepdims=True)
    T_mean = np.mean(T, axis=1, keepdims=True)
    P_std = np.std(P, axis=1, keepdims=True)
    T_std = np.std(T, axis=1, keepdims=True)
    
    P_normalized = (P - P_mean) / P_std
    T_normalized = (T - T_mean) / T_std
    C = np.dot(P_normalized, T_normalized.T) / P.shape[1]

    id_acc = np.zeros(n)
    for i in range(n):
        id_acc[i] = (C[i, i] >= C[i]).sum()  
        id_acc[i] = id_acc[i] / (n - 1)
    
    return np.mean(id_acc)


In [11]:
def evaluate_and_log(loader, brain_model):
    results = organize_results(loader, brain_model)
    
    print("Starting evaluation...")
    for subject_index, subject_data in results.items():
        print(f"Evaluating metrics for subject {subject_index}...")
        
        y_pred = subject_data["y_pred"]
        y_gt = subject_data["y_gt"]
        gt_images = subject_data["gt_images"]

        y_pred_np = y_pred.cpu().numpy()
        y_gt_np = y_gt.cpu().numpy()

        # Compute similarity matrix
        similarity_matrix = cosine_similarity(y_pred_np, y_gt_np)
        print(f"Computed similarity matrix for subject {subject_index}.")

        # Calculate Top-1 and Top-5 accuracy
        top1_acc = (np.argmax(similarity_matrix, axis=1) == np.arange(len(y_gt_np))).mean()
        top5_acc = np.mean([1 if i in np.argsort(-similarity_matrix[i])[:5] else 0 for i in range(len(y_gt_np))])
        print(f"Top-1 Accuracy: {top1_acc:.4f}, Top-5 Accuracy: {top5_acc:.4f} for subject {subject_index}.")

        # Identification accuracy
        id_acc = identification_accuracy_fast(y_pred_np, y_gt_np)
        print(f"Identification accuracy for subject {subject_index}: {id_acc:.4f}")

        # Log metrics for this subject
        wandb.log({
            f"subject_{subject_index}_top1_acc": top1_acc,
            f"subject_{subject_index}_top5_acc": top5_acc,
            f"subject_{subject_index}_identification_accuracy": id_acc,
        })

        # Log a sample of 10 images and top-5 retrievals for each subject
        sample_indices = np.random.choice(len(y_gt_np), 10, replace=False)
        subject_table = wandb.Table(columns=["Stimulus Image", "Top 5 Retrieved Images"])

        for i in sample_indices:
            sim_scores = similarity_matrix[i]
            top5_indices = np.argsort(-sim_scores)[:5]
            top5_images = [wandb.Image(gt_images[idx]) for idx in top5_indices]  # Convert to wandb.Image
            stimulus_image = wandb.Image(gt_images[i])  # Convert stimulus to wandb.Image
            subject_table.add_data(stimulus_image, top5_images)
        
        print(f"Logged top-5 retrievals for subject {subject_index}.")
        
        # Log the table for each subject
        wandb.log({f"subject_{subject_index}_Top5_Retrievals": subject_table})

    print("Evaluation complete. Results loaded to wandb.")

In [12]:
activation_functions = {
    'ReLU': nn.ReLU,
    'Identity': nn.Identity
}


def train_model():
    # Initialize the wandb run
    wandb.init()

    # Extract hyperparameters from wandb config
    config = wandb.config
        
    train_loader = DataLoader(train_dataset, batch_size=config.BS, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=config.BS, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=config.BS, shuffle=True,num_workers=4)

    brain_model = ContrastiveModel(num_input_channels= 10000,
                                   base_channel_size=config.base_channel_size, 
                                   hidden_dims=config.hidden_dims,
                                   latent_dim=config.latent_dim,
                                   act_fn=activation_functions[config.act_fn],
                                   loss_type=config.loss_type,
                                   lr = config.lr,
                                   wd = config.wd,
                                   alpha=config.alpha)
    
    # Set up early stopping to monitor 'val_loss'
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=1,verbose=True, mode='min')             # 'min' because we want to minimize val_loss
    wandb_logger = WandbLogger()  # Logs the model and metrics to wandb


    # Create a unique checkpoint directory based on the run name or ID
    run_name = wandb.run.name or wandb.run.id  # Use run name if available, otherwise use run ID
    checkpoint_dir = os.path.join(data_path, "models", run_name)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Model checkpoint configuration
    checkpoint_callback = ModelCheckpoint(monitor='val_loss',dirpath=checkpoint_dir,filename='brain_model-{epoch:02d}-{val_loss:.2f}',save_top_k=3,mode='min',)
        
        
        

    # Initialize trainer with logger

    trainer = pl.Trainer(max_epochs=20, devices=[1], callbacks=[early_stop_callback,checkpoint_callback],logger=wandb_logger ) # Add the wandb logger here

    trainer.fit(brain_model, train_loader, val_loader)

    # Load the best model from the checkpoint
    # best_model = ContrastiveModel.load_from_checkpoint(checkpoint_callback.best_model_path)
    # best_model.eval()

    ## ADD OTHER EVALUATIONS

    evaluate_and_log(val_loader,brain_model)

    # train_fmri_embeddings, train_img_embeddings = get_embeddings(brain_model, train_loader)
    # val_fmri_embeddings, val_img_embeddings = get_embeddings(brain_model, test_loader)
    # test_fmri_embeddings, test_img_embeddings = get_embeddings(brain_model, test_loader)




In [13]:
wandb.agent(sweep_id, function=train_model, count=20)

wandb: Agent Starting Run: zr12hlzj with config:
wandb: 	BS: 2048
wandb: 	act_fn: Identity
wandb: 	alpha: 0.8
wandb: 	base_channel_size: [2048, 1024]
wandb: 	hidden_dims: [2048, 1024, 512]
wandb: 	latent_dim: 768
wandb: 	loss_type: contrastive
wandb: 	lr: 0.0001
wandb: 	temperature: 0.1
wandb: 	wd: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matteoferrante. Use `wandb login --relogin` to force relogin


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performanc

Sanity Checking: 0it [00:00, ?it/s]

/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.919


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 1 records. Best score: 0.919. Signaling Trainer to stop.
100%|██████████| 2/2 [00:13<00:00,  6.63s/it]


Starting evaluation...
Evaluating metrics for subject 1...
Computed similarity matrix for subject 1.
Top-1 Accuracy: 0.0252, Top-5 Accuracy: 0.1385 for subject 1.
Identification accuracy for subject 1: 0.8287
Logged top-5 retrievals for subject 1.
Evaluating metrics for subject 2...
Computed similarity matrix for subject 2.
Top-1 Accuracy: 0.0403, Top-5 Accuracy: 0.1385 for subject 2.
Identification accuracy for subject 2: 0.8316
Logged top-5 retrievals for subject 2.
Evaluating metrics for subject 3...
Computed similarity matrix for subject 3.
Top-1 Accuracy: 0.0277, Top-5 Accuracy: 0.1108 for subject 3.
Identification accuracy for subject 3: 0.8304
Logged top-5 retrievals for subject 3.
Evaluating metrics for subject 4...
Computed similarity matrix for subject 4.
Top-1 Accuracy: 0.0428, Top-5 Accuracy: 0.1511 for subject 4.
Identification accuracy for subject 4: 0.8389
Logged top-5 retrievals for subject 4.
Evaluation complete. Results loaded to wandb.


epoch,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▅▆▆▇▇▇▇██
subject_1_identification_accuracy,▁
subject_1_top1_acc,▁
subject_1_top5_acc,▁
subject_2_identification_accuracy,▁
subject_2_top1_acc,▁
subject_2_top5_acc,▁
subject_3_identification_accuracy,▁
subject_3_top1_acc,▁
subject_3_top5_acc,▁
subject_4_identification_accuracy,▁
